# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,18.69,83,100,1.34,US,2024-08-09 03:11:11
1,1,invercargill,-46.4000,168.3500,9.45,79,0,2.24,NZ,2024-08-09 03:12:08
2,2,la ronge,55.1001,-105.2842,14.77,56,35,1.19,CA,2024-08-09 03:12:10
3,3,bethel,41.3712,-73.4140,20.01,90,100,0.45,US,2024-08-09 03:12:12
4,4,edinburgh of the seven seas,-37.0676,-12.3116,14.18,90,100,18.45,SH,2024-08-09 03:12:13


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True, 
    tiles='OSM', 
    color='City',  # Color by city name or another category if you prefer
    size='Humidity',  # Size of the point corresponds to humidity
    hover_cols=['City', 'Country', 'Humidity']  # Columns to display when hovering over a point
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 70) & 
    (city_data_df["Max Temp"] > 20) &  
    (city_data_df['Wind Speed'] < 10) & 
    (city_data_df['Cloudiness'] < 20)
]
# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,san patricio,28.0170,-97.5169,30.50,68,0,4.27,US,2024-08-09 03:12:17
13,13,carnarvon,-24.8667,113.6333,22.26,42,0,4.89,AU,2024-08-09 03:12:26
16,16,cabedelo,-6.9811,-34.8339,23.37,83,4,4.99,BR,2024-08-09 03:12:30
27,27,cururupu,-1.8283,-44.8683,24.84,83,6,1.48,BR,2024-08-09 03:12:47
33,33,ha'il,27.5219,41.6907,25.01,19,0,1.03,SA,2024-08-09 03:12:55


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
7,san patricio,US,28.0170,-97.5169,68,
13,carnarvon,AU,-24.8667,113.6333,42,
16,cabedelo,BR,-6.9811,-34.8339,83,
27,cururupu,BR,-1.8283,-44.8683,83,
33,ha'il,SA,27.5219,41.6907,19,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000  # Radius in meters
api_key = geoapify_key  # Assuming you have your Geoapify API key stored in `geoapify_key`
params = {
    'categories': 'accommodation.hotel',  # Searching for hotels
    'limit': 1,  # Limit to the first hotel found
    'apiKey': api_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary --- name_address = # YOUR CODE HERE checar
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
san patricio - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
cabedelo - nearest hotel: No hotel found
cururupu - nearest hotel: No hotel found
ha'il - nearest hotel: فندق كلاسيك
laguna - nearest hotel: Holiday Inn Express & Suites
pinoso - nearest hotel: Hostal La Canada
fort stockton - nearest hotel: La Quinta Inn
praia da vitoria - nearest hotel: Salles
sokoni - nearest hotel: No hotel found
brownwood - nearest hotel: Best Western Plus Riata
tucacas - nearest hotel: Posada Vida Playera
charters towers - nearest hotel: No hotel found
verkhnevilyuysk - nearest hotel: No hotel found
road town - nearest hotel: No hotel found
alice springs - nearest hotel: Aurora Alice Springs
kyongwon - nearest hotel: No hotel found
balaklava - nearest hotel: Рыбацкая слобода
buka - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
mount isa - nearest hotel: Ibis Styles
ribeira grande - nearest hotel: Hospedaria JSF
hotan - nearest

,City,Country,Lat,Lng,Humidity,Hotel Name
7,san patricio,US,28.0170,-97.5169,68,No hotel found
13,carnarvon,AU,-24.8667,113.6333,42,No hotel found
16,cabedelo,BR,-6.9811,-34.8339,83,No hotel found
27,cururupu,BR,-1.8283,-44.8683,83,No hotel found
33,ha'il,SA,27.5219,41.6907,19,فندق كلاسيك


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True, 
    tiles='OSM', 
    color='City',  # Color by city name or another category if you prefer
    size='Humidity',  # Size of the point corresponds to humidity
    hover_cols=['City', 'Country', 'Humidity', 'Hotel Name']  # Include Hotel Name and Country in the hover message
)
# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)